# Tachyonic condensation · Tube condensation walkthrough

This notebook exercises `code/tachyonic_condensation/condense_tube.py`. When cylinder modes are available we project the quartic coupling, compute condensate amplitudes, and summarise the diagonal mass shifts.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from code.tachyonic_condensation import cylinder_modes, condense_tube
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from code.tachyonic_condensation import cylinder_modes, condense_tube


if not getattr(cylinder_modes, '_HAVE_SCIPY', False):
    print('SciPy is required for the tube condensation demo; install scipy to continue.')
else:
    roots = cylinder_modes.compute_kappas(R=3.0, mu=1.0, c=1.0, ell_max=2, num_brackets=64, tol=1e-6)
    modes = [condense_tube.ModeEntry(ell=int(r['ell']), kappa=float(r['kappa']), k_in=float(r['k_in']), k_out=float(r['k_out'])) for r in roots]
    N4 = condense_tube.build_quartic_diagonal(R=3.0, modes=modes, lam=0.5, c=1.0)
    v = condense_tube.find_condensate_diagonal(R=3.0, modes=modes, N4=N4, c=1.0)
    M2 = condense_tube.mass_matrix_diagonal(modes=modes, N4=N4, v=v, c=1.0)
    print(preview_json({"modes": len(modes), "N4": N4, "v": v, "M2": M2}))


## Notebook checklist

* The diagonal λ approximation follows the same assumptions as the script; extend it with off-diagonal elements if needed.
* Tune `lam`, `R`, and the maximum ℓ to explore different condensation regimes.
* Use the returned dictionaries (`N4`, `v`, `M2`) as building blocks for energy scans or visual summaries.
